**CONFIGURATION**

In [0]:
%sql
USE CATALOG main;
CREATE VOLUME IF NOT EXISTS main.meta.checkpoints;

In [0]:
%sql
select * from main.meta.batch_config

In [0]:
%sql 

CREATE TABLE IF NOT EXISTS main.meta.batch_config ( 
  source_id STRING, 
  source_name STRING, 
  modality STRING, 
  target_catalog STRING, 
  target_schema STRING, 
  target_table STRING, 
  partition_columns ARRAY<STRING>, 
  load_type STRING, 
  run_order INT, 
  enabled BOOLEAN, 
  schedule STRING, 
  created_at TIMESTAMP, 
  updated_at TIMESTAMP, 
  file_format VARCHAR(50), 
  target_file_format VARCHAR(50),
  PRIMARY KEY (source_id) 
) 

USING DELTA; 

**Batch Config**

In [0]:
%sql
#drop table main.meta.batch_config

In [0]:
%sql
INSERT INTO main.meta.batch_config (
   source_id, source_name, modality, target_catalog, target_schema, target_table,
   partition_columns, load_type, run_order, enabled, schedule, created_at, updated_at, file_format, target_file_format
)
VALUES
  ('src_101', 'File CSV', 'csv', 'main', 'meta', 'main.meta.bronze_batch',
  ARRAY('region', 'date'), 'full', 1, TRUE, 'daily', CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP(), 'csv','parquet'),
  ('src_201', 'File Json', 'json', 'main', 'meta', 'main.meta.bronze_batch',
  ARRAY('region', 'date'), 'full', 1, FALSE, 'daily', CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP(), 'json','delta')

In [0]:
%sql
update main.meta.batch_config
set source_name = 'data.json', file_format = 'json', modality = 'json'
where source_id = 'src_101'

**API Config**

In [0]:
%sql
INSERT INTO main.meta.api_config (
   source_id, source_name, modality, target_catalog, target_schema, target_table,
   partition_columns, load_type, run_order, enabled, schedule, created_at, updated_at
)
VALUES (
   'src_201', 'API_Sales', 'API', 'main', 'meta', 'main.meta.bronze_API',
   array('date'), 'incremental', 1, true, 'daily', current_timestamp(), current_timestamp()
);
---- Function to be updated for API

**Streaming Config**

In [0]:
%sql
ALTER TABLE main.meta.api_config
ADD COLUMNS (target_file_format VARCHAR(50));

In [0]:
%sql
UPDATE main.meta.api_config
SET target_file_format = 'parquet'
where source_id= 'src_201'

In [0]:
et%sql
INSERT INTO main.meta.streaming_config (
   source_id, source_name, modality, file_format, target_catalog, target_schema, target_table,
   partition_columns, load_type, run_order, enabled, schedule, created_at, updated_at
)
VALUES
  ('src_301', 'new data', 'streaming','csv', 'main', 'meta', 'main.meta.bronze_streaming',
    ARRAY('region', 'date'), 'append', 1, TRUE, 'hourly', CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP());

    ---- Function to be updated for Streaming

**JDBC Config**

In [0]:
%sql
INSERT INTO main.meta.jdbc_config (
   source_id, source_name, modality, target_catalog, target_schema, target_table,
   partition_columns, load_type, run_order, enabled, schedule, created_at, updated_at
)
VALUES (
   'src_401', 'Postgres JDBC', 'jdbc', 'main', 'meta', 'main.bronze_JDBC',
   ARRAY('date'), 'incremental', 4, TRUE, 'daily', CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP()
);

---- Function to be updated for jdbc

**Connection Config**

In [0]:
%sql
INSERT INTO main.meta.connection_config (
  source_id, connection_name, connection_type, source_path,
  jdbc_url, jdbc_user, jdbc_password, api_url, api_method, api_headers,
  api_auth_type, secret_scope, secret_key, enabled, created_at, updated_at
)
VALUES (
  'src_101', 'CSV Volume', 'volume',
  '/Volumes/main/meta/inputfiles/batch_files',
  NULL, NULL, NULL, NULL, NULL, NULL,
  NULL, NULL, NULL, TRUE, CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP()
),
 (
  'src_201', 'OpenMeteo API', 'api', NULL,
  NULL, NULL, NULL,
  'https://jsonplaceholder.typicode.com/posts',
  'GET', NULL,
  'None', NULL, NULL,
  TRUE, CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP()
),
  (
   'src_301', 'new data', 'streaming',
   '/Volumes/main/meta/inputfiles/streaming_files',
    NULL, NULL, NULL, NULL, NULL, NULL,
    NULL, NULL, NULL, TRUE, CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP()
  ),
  ('src_401', 'Postgres JDBC', 'jdbc', NULL,
    'jdbc:postgresql://your-host:5432/your-db', 'your_user', 'your_password',
    NULL, NULL, NULL,
    NULL, NULL, NULL,
    TRUE, CURRENT_TIMESTAMP(), CURRENT_TIMESTAMP()
  );

  -- No changes needed

**Delete Satements**

In [0]:

spark.sql("DELETE FROM main.meta.batch_config")

In [0]:
%python

spark.sql("DELETE FROM main.meta.api_config")
spark.sql("DELETE FROM main.meta.streaming_config")
spark.sql("DELETE FROM main.meta.jdbc_config")



In [0]:
%sql
UPDATE main.meta.batch_config
SET
  file_format = 'csv'
WHERE source_id = 'src_301';

In [0]:
%sql
UPDATE main.meta.streaming_config
SET
  source_path = '/Volumes/main/meta/inputfiles/streaming_files',
  file_format = 'csv'
WHERE source_id = 'src_301';

--RUn after updating streaming 